# Building and Pushing Custom Docker Images to Amazon ECR

In the first notebook, we will focus on the process of creating a custom Docker image. This will involve selecting a base image, installing necessary dependencies, and configuring the environment to suit your specific ML requirements. We'll then guide you through the process of pushing this custom image to Amazon Elastic Container Registry (ECR), a fully managed Docker container registry that makes it easy to store, manage, share, and deploy your container images.

https://docs.aws.amazon.com/sagemaker/latest/dg/notebooks-available-images.html

In [1]:
import os
import json
import boto3
import sagemaker

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
!mkdir container

## Creating Requirements Files for Custom Environments

We create two distinct requirements files, each tailored for a specific custom environment in our Docker container. These files list the Python libraries that will be installed in each environment.

To create these files, we use the `%%writefile` magic command in Jupyter Notebook.

In [3]:
%%writefile container/requirements_env1.txt
Faker

Writing container/requirements_env1.txt


In [4]:
%%writefile container/requirements_env2.txt
matplotlib

Writing container/requirements_env2.txt


## Constructing the Dockerfile for Custom Environments

This section involves creating a [Dockerfile](https://docs.docker.com/engine/reference/builder/#dockerfile-reference) to define the custom Docker image. This Dockerfile is based on the official Python 3.11 image and includes the setup for two distinct Python virtual environments. These environments correspond to the two sets of requirements we defined earlier.

> In this Dockerfile, we emphasize the installation and configuration of **ipykernel** in each virtual environment. This step is crucial because Amazon SageMaker Studio utilizes these kernels to interface with the Docker image. When SageMaker Studio users select a custom image for their Jupyter notebooks, it identifies the available Python environments through these kernels. 

In [5]:
%%writefile container/Dockerfile
FROM python:3.11 as base

# Create virtual environments
RUN python3 -m venv /env1
RUN python3 -m venv /env2

# Environment 1 setup
COPY requirements_env1.txt /
RUN /env1/bin/pip install --no-cache-dir ipykernel
RUN /env1/bin/pip install -r /requirements_env1.txt
RUN /env1/bin/python -m ipykernel install --name=env1 --display-name "Python 3.11 [pip env: Faker]"

# Environment 2 setup
COPY requirements_env2.txt /
RUN /env2/bin/pip install --no-cache-dir ipykernel
RUN /env2/bin/pip install -r /requirements_env2.txt
RUN /env2/bin/python -m ipykernel install --name=env2 --display-name "Python 3.11 [pip env: matplotlib]"

Writing container/Dockerfile


To maintain the efficiency and cleanliness of your Docker environment, it's important to occasionally remove unused Docker objects. This is where the command `!docker system prune -af` comes into play.

+ `docker system prune``: This command removes unused Docker objects, such as containers, networks, images (both dangling and unreferenced), and optionally, volumes.
+ `-a` or `--all`: This flag removes all unused images, not just dangling ones.
+ `-f` or `--force`: This automatically confirms the removal without prompting for confirmation.

In [ ]:
!docker system prune -af

The following script builds a Docker image from the Dockerfile and pushes it to Amazon ECR.

**Script Breakdown**

1. Setting Parameters: The script sets the repository and image names, and identifies the AWS region and account ID to construct the full name of the ECR repository.

2. Checking for ECR Repository: It checks if the specified ECR repository exists. If not, it creates a new repository.

3. ECR Login: The script logs into ECR using aws ecr get-login-password, which is necessary for pushing images to the repository.

4. Building the Docker Image: The Docker image is built from the Dockerfile located in the container directory. The image is tagged with both a simple name and the full ECR repository name.

5. Pushing to ECR: Finally, the script pushes the Docker image to the ECR repository.

Before proceeding with building and pushing Docker images to Amazon ECR, ensure you have an IAM role with the appropriate permissions for ECR operations. This role should include a policy that allows actions related to managing and uploading images to your ECR repository:

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "ecr:BatchCheckLayerAvailability",
                "ecr:BatchGetImage",
                "ecr:CompleteLayerUpload",
                "ecr:GetDownloadUrlForLayer",
                "ecr:InitiateLayerUpload",
                "ecr:PutImage",
                "ecr:UploadLayerPart",
                "ecr:ListImages"
            ],
            "Resource": "arn:aws:ecr:region:account_id:repository/sagemaker-custom-images"
        }
    ]
}

```


In [ ]:
%%sh

cd container

# Parameters
REPO_NAME=sagemaker-custom-images
IMAGE_NAME=test-image

# Region, defaults to us-east-1
REGION=$(aws configure get region)
REGION=${REGION:-us-east-1}

ACCOUNT_ID=$(aws sts get-caller-identity --query Account --output text)
FULL_NAME="${ACCOUNT_ID}.dkr.ecr.${REGION}.amazonaws.com/${REPO_NAME}:${IMAGE_NAME}"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${REPO_NAME}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${REPO_NAME}" > /dev/null
fi

# Login to ECR
aws ecr get-login-password --region ${REGION} | docker login --username AWS --password-stdin ${FULL_NAME}

# Build and push the image
docker build . -t ${IMAGE_NAME} -t ${FULL_NAME}

docker push ${FULL_NAME}

If you have successfully completed all the steps, your image should now be available in your ECR repository.

![Image in ECR](img/ecr_image.png "Image in ECR")